In [7]:
import numpy as np

In [8]:
class Node:
    """Node in decision tree
    """
    def __init__(self, feature=None, threshold=None, left=None, right=None, var_red=None, value=None):
        # Internal node
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red # Variance reduction
        
        # Leaf node 
        self.value = value

In [9]:
class DecisionTreeRegressor:
    """Decision tree regressor
    
    Attributes:
    -----------
    max_depth: int, default to None
        The maximum depth of the tree

    min_samples_split: int, default=2
        The minimum number of samples required to split an internal node
    """
    def __init__(self, max_depth=3, min_samples_split=2):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
    def _calulate_leaf_value(self, y):
        return np.mean(y)
        
    def _variance_reduction(self, parent, left, right):
        left_weight = len(left) / len(parent)
        right_weight = len(right) / len(parent)
        var_red = np.var(parent) - left_weight * np.var(left) - right_weight * np.var(right)
        return var_red
    
    def _split(self, X, y, feature, threshold):
        left = (X[:, feature] <= threshold)
        right = (X[:, feature] > threshold)
        X_left = X[left]
        X_right = X[right]
        y_left = y[left]
        y_right = y[right]
        return X_left, y_left, X_right, y_right
    
    def _get_best_split(self, X, y):
        """Find the best split
        """
        best_split = dict()
        n_features = X.shape[1]
        max_var_red = -float("inf")
        
        # Loop over all features
        for feature in range(n_features):
            feature_values = X[:, feature]
            possible_thresholds = np.unique(feature_values)
            
            # Loop over all possible features
            for threshold in possible_thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold) # Current split
                if len(y_left) > 0 and len(y_right) > 0: 
                    current_var_red = self._variance_reduction(y, y_left, y_right)
                    if current_var_red > max_var_red:
                        max_var_red = current_var_red
                        best_split['feature'] = feature 
                        best_split['threshold'] = threshold
                        best_split['X_left'] = X_left
                        best_split['y_left'] = y_left
                        best_split['X_right'] = X_right
                        best_split['y_right'] = y_right
                        best_split['var_red'] = current_var_red
        return best_split
    
    def _build_tree(self, X, y, current_depth):
        """Build the tree recursively
        """
        n_samples, n_features = X.shape

        # Stoppig condition
        if n_samples >= self.min_samples_split and  current_depth < self.max_depth:
            best_split = self._get_best_split(X, y) # Find the best split
            if best_split['var_red'] >= 0:
                left_subtree = self._build_tree(best_split['X_left'], best_split['y_left'], current_depth+1) # Build left sub-tree
                right_subtree = self._build_tree(best_split['X_right'], best_split['y_right'], current_depth+1) # Build rigth sub-tree
                
                return Node(best_split['feature'], best_split['threshold'], 
                           left_subtree, right_subtree, best_split['var_red'])
            
        leaf_value = self._calulate_leaf_value(y)
        return Node(value=leaf_value)
        
    def fit(self, X, y):
        """Train the data to build tree
        """
        self.root = self._build_tree(X, y, current_depth=0)
        
    def _make_prediction(self, tree, x):
        if tree.value != None:
            return tree.value
        feature_x = x[tree.feature]
        if feature_x <= tree.threshold:
            return self._make_prediction(tree.left, x)
        else:
            return self._make_prediction(tree.right, x)
        
    def predict(self, X):
        """Predict labels for new data
        """
        predictions = [self._make_prediction(self.root, x) for x in X]
        return np.array(predictions, dtype=np.int)

In [10]:
# Test
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
reg1 = DecisionTreeRegressor()
reg1.fit(X_train, y_train)
y_pred = reg1.predict(X_test)
mean_squared_error(y_pred, y_test)

22.26417322834646

In [15]:
from sklearn import tree
reg2 = tree.DecisionTreeRegressor()
reg2.fit(X_train, y_train)
y_pred = reg2.predict(X_test)
mean_squared_error(y_pred, y_test)

17.082755905511807